Copyright © 2021, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Unsupervised identification of disease similarity and drug compound similarity 

This demo is based on the dataset available from https://github.com/hetio/hetionet.

Hetionet is a knowledge graph that encodes known information in biology and medicine.
   
The data model consists of various node and link types.
   
The goal of this demo is to predict disease resemblance and compound resemblance using an unsupervised approach.
   
All the "resembles" links between Disease node pairs and Compound node pairs in the data set are considered unknown for the sake of this demo. The task is to predict these links based on the other information in the network.

This task is considered unsupervised, because none of the links to be predicted are made available to the model during training.

The sparsity of the prediction task makes it a difficult machine learning challenge, but the approach given in this notebook uses network projection to achieve highly accurate results.

This type of unsupervised analysis could be useful in a drug repurposing study: one could identify plausible candidates for a clinical trial based on the following:
- which compounds are most similar to a compound X that is known to be an effective treatment of disease Y?
- which compounds are known to be effective in treating the diseases that are most similar to disease Y?

# Imports

In [1]:
import swat
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),r"../../../common/python"))
import visualization as viz
import cas_connection as cas

# CAS Connection

In [2]:
caslib = 'myData'
subdir = 'data/hetionet/'
s      =  cas.reconnect(caslib=caslib, datasubdir=subdir)
s.loadActionSet('network')
s.loadActionSet('fedsql')

NOTE: 'myData' is now the active caslib.
NOTE: Cloud Analytic Services added the caslib 'myData'.
NOTE: Added action set 'network'.
NOTE: Added action set 'fedsql'.


[actionset]

 'fedsql'

+ Elapsed: 0.00146s, user: 0.00129s, sys: 0.00014s, mem: 0.239mb

## Options
For disease link prediction, select "Disease". For drug compound link prediction, select "Compound"

In [3]:
keep_type = "Disease"

# Read in the Knowledge Graph

In [4]:
s.table.loadTable(
    caslib        = caslib,
    path          = "hetionet-v1.0-nodes.tsv",
    casout        = {"name":"nodes", "replace":True},
    importOptions = { "fileType":"CSV", "delimiter":"\t",
        "vars":[
            {"name":"node", "type":"CHAR", "length":48},
            {"name":"name", "type":"CHAR", "length":64},
            {"name":"type", "type":"CHAR", "length":24}
        ]
    }
)

NOTE: Cloud Analytic Services made the file hetionet-v1.0-nodes.tsv available as table NODES in caslib myData.


[caslib]

 'myData'

[tableName]

 'NODES'

[casTable]

 CASTable('NODES', caslib='myData')

+ Elapsed: 0.0268s, user: 0.0532s, sys: 0.0134s, mem: 42.2mb

In [5]:
s.table.loadTable(
    caslib        = caslib,
    path          = "hetionet-v1.0-edges.sif",
    casout        = {"name":"links", "replace":True},
    importOptions = { "fileType":"CSV", "delimiter":"\t",
        "vars":[
            {"name":"from", "type":"CHAR", "length":48},
            {"name":"type", "type":"CHAR", "length":24},
            {"name":"to", "type":"CHAR", "length":48}
        ]
    }
)

NOTE: Cloud Analytic Services made the file hetionet-v1.0-edges.sif available as table LINKS in caslib myData.


[caslib]

 'myData'

[tableName]

 'LINKS'

[casTable]

 CASTable('LINKS', caslib='myData')

+ Elapsed: 0.657s, user: 1.97s, sys: 0.324s, mem: 155mb

## Inspect Data Model

In [6]:
node_type_counts = s.CASTable("nodes")["type"].value_counts()
node_type_counts

Gene                   20945
Biological Process     11381
Side Effect             5734
Molecular Function      2884
Pathway                 1822
Compound                1552
Cellular Component      1391
Symptom                  438
Anatomy                  402
Pharmacologic Class      345
Disease                  137
dtype: int64

In [7]:
n_diseases  = node_type_counts["Disease"]
n_compounds = node_type_counts["Compound"]

In [8]:
link_type_counts = s.CASTable("links")["type"].value_counts()
link_type_counts

GpBP    559504
AeG     526407
Gr>G    265672
GiG     147164
CcSE    138944
AdG     102240
AuG      97848
GpMF     97222
GpPW     84372
GpCC     73566
GcG      61690
CdG      21102
CuG      18756
DaG      12623
CbG      11571
DuG       7731
DdG       7623
CrC       6486
DlA       3602
DpS       3357
PCiC      1029
CtD        755
DrD        543
CpD        390
dtype: int64

In [9]:
n_drd  = link_type_counts["DrD"]
n_crc = link_type_counts["CrC"]

## Hide Ground Truth Links

In [10]:
s.datastep.runCode(
    code="""
        data linksTrain;
        set links;
        if type ne "DrD" and type ne "CrC";
        run;
    """
)

[InputCasTables]

    casLib   Name     Rows  Columns                            casTable
 0  myData  links  2250197        3  CASTable('links', caslib='myData')

[OutputCasTables]

    casLib        Name     Rows  Columns  Append Promoted  \
 0  myData  linksTrain  2243168        3     NaN        N   
 
                                   casTable  
 0  CASTable('linksTrain', caslib='myData')  

+ Elapsed: 0.142s, user: 0.847s, sys: 0.291s, mem: 69.8mb

Verify that there are no 'resembles' links

In [11]:
types_train = s.CASTable("linksTrain")["type"].unique()
print(types_train)
print(f"Has Disease-resembles-Disease links: {'DrD' in types_train}")
print(f"Has Compound-resembles-Compound links: {'CrC' in types_train}")

['AdG' 'AeG' 'AuG' 'CbG' 'CcSE' 'CdG' 'CpD' 'CtD' 'CuG' 'DaG' 'DdG' 'DlA'
 'DpS' 'DuG' 'GcG' 'GiG' 'GpBP' 'GpCC' 'GpMF' 'GpPW' 'Gr>G' 'PCiC']
Has Disease-resembles-Disease links: False
Has Compound-resembles-Compound links: False


# Projections

This function performs a projection that infers links between two nodes based on the common neighborhoods of those nodes.

   Assumption: Links of the *link_type* type express a bipartite subgraph of the graph represented by
   the *nodes* and *links* tables.

   Parameters:
   
   <b>nodes</b> - the name of the nodes table. Assumes the "type" column identifies the node type<br>
   <b>links</b> - the name of the links table. Assumes the "type" column identifies the link type<br>
   <b>from_node_type</b> - the node type of the primary partition. Output will be inferred links between pairs of nodes of this type<br>
   <b>to_node_type</b> - the node type of the secondary partition. Output inferred links will be based on common neighbors of this type<br>
   <b>link_type</b> - the link type to utilize. Output will be based only upon input graph links of this type<br>
   <b>out_nodes</b> - the output table that gives the list of nodes (they will all be of type <i>from_node_type</i>)<br>
   <b>out_links</b> - the output table that gives the inferred links. Link strength can be quantified using the adamicAdar, cosine, jaccard, and commonNeighbors similarity scores<br>


In [12]:
def project(nodes, links, from_node_type, to_node_type, link_type, out_nodes, out_links, append_links=None):
    s.network.projection(
        links       = {"name":links, "where":f"type EQ {link_type}"},
        nodes       = {"name":nodes,
            "computedVars": {"partitionFlag"}, # Specifies the node roles in projection
            "computedVarsProgram": f"""if type = '{from_node_type}'
                then partitionFlag = 1;
                else if type = '{to_node_type}' 
                then partitionFlag = 0;
                else partitionFlag = .;"""
        },
        nodesVar    = {"vars":["name", "type"]},
        partition = "partitionFlag",
        outProjectionLinks = {"name":"out_links_", "replace":True},
        outProjectionNodes = {"name":f"{out_nodes}", "replace":True},
#         Output-weighting column options
        adamicAdar = True,
        cosine = True,
        jaccard = True,
        commonNeighbors = True
    )
    
    s.datastep.runCode(
        code=f"""
            data {out_links};
               set {append_links} out_links_(in=newLink);
               length type VARCHAR(34);
               if newLink then do;
                  type = "SAME " || "{link_type}";
               end;
            run;
        """
    )

In [13]:
def get_parameters(keep_type):
    resembles_link_type = None
    fetch_cutoff_weight = None
    if keep_type == "Disease":
        resembles_link_type = "DrD"
        fetch_cutoff_weight = 0.1
    elif keep_type == "Compound":
        resembles_link_type = "CrC"
        fetch_cutoff_weight = 0.1
    return resembles_link_type, fetch_cutoff_weight

resembles_link_type, fetch_cutoff_weight = get_parameters(keep_type)

In [14]:
def do_projections(keep_type):
    if keep_type == "Disease":
        project("nodes", "linksTrain", keep_type, "Compound", "'CtD'", "projNodes", "projLinks", append_links="");
        project("nodes", "linksTrain", keep_type, "Compound", "'CpD'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Gene", "'DdG'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Gene", "'DuG'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Gene", "'DaG'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Anatomy", "'DlA'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Symptom", "'DpS'", "projNodes", "projLinks", append_links="projLinks");
    elif keep_type == "Compound":
        project("nodes", "linksTrain", keep_type, "Disease", "'CtD'", "projNodes", "projLinks", append_links="");
        project("nodes", "linksTrain", keep_type, "Disease", "'CpD'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Gene", "'CdG'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Gene", "'CuG'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Gene", "'CbG'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Pharmacologic Class", "'PCiC'", "projNodes", "projLinks", append_links="projLinks");
        project("nodes", "linksTrain", keep_type, "Side Effect", "'CcSE'", "projNodes", "projLinks", append_links="projLinks");
    
do_projections(keep_type)

NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 755.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.05 (cpu: 0.06) seconds.
NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 390.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.05 (cpu: 0.13) seconds.
NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 7623.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.06 (cpu: 0.14) seconds.
NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 7731.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.05 (cpu: 0.11) seconds.
NOTE: The number of nodes in the input gra

# Join Nodes Data

In [15]:
def join_nodes_data():
    s.fedsql.execDirect(
        query="""
            create table projLinksJoined {options replace=true} as
            select a.from, a.to, b.name as "fromName", c.name as "toName", a.adamicAdar, a.type, a.jaccard, a.cosine, a.commonNeighbors
            from projLinks a
            join nodes b
            on a.from = b.node
            join nodes c
            on a.to = c.node
            ;
        """
    )

    s.fedsql.execDirect(
        query="""
            create table projLinksGrouped {options replace=true} as
            select MIN(a.from) as "from",
                  MIN(a.to) as "to",
                  fromName, toName,
                  SUM(adamicAdar) as "adamicAdarSum",
                  SUM(jaccard) as "jaccardSum",
                  SUM(cosine) as "cosineSum",
                  sum(commonNeighbors) as "commonNeighborsSum",
                  /* We have no reason to prefer one weighting methodology over another */
                  /* so here, we simply combine all four via geometric mean */
                  POWER(SUM(adamicAdar)*SUM(jaccard)*SUM(cosine)*sum(commonNeighbors),0.25) as "weight"
            from projLinksJoined as a
            group by fromName, toName
        """
    )
    
join_nodes_data()

NOTE: Table PROJLINKSJOINED was created in caslib myData with 17161 rows returned.
NOTE: Table PROJLINKSGROUPED was created in caslib myData with 7542 rows returned.


## Determine the Range of Weights

In [16]:
def get_min_max_weights():
    min_weight = s.CASTable("projLinksGrouped")["weight"].min()
    max_weight = s.CASTable("projLinksGrouped")["weight"].max()
    return min_weight, max_weight

min_weight, max_weight = get_min_max_weights()

# Evaluation

Gather the Ground truth "resembles" links

Use a right join to mark which inferred links from projLinks are also ground truth links

In [17]:
def join_ground_truth(resembles_link_type, fetch_cutoff_weight, min_weight, max_weight):
    s.datastep.runCode(
        code=f"""
            data linksRes;
               set links(where=(type="{resembles_link_type}"));
               if to < from then do;
                  temp=to;
                  to=from;
                  from=temp;
               end;
               drop temp;
            run;
        """
    )
    fetch_cutoff = fetch_cutoff_weight*(max_weight-min_weight)
    s.fedsql.execDirect(
        query=f"""
            create table linksResJoinedR {{options replace=true}} as
           select a.*, b.weight as "weight", c.name as "fromName", d.name as "toName"
           from linksRes a
           right join projLinksGrouped b
           on a.from = b.from and a.to = b.to
           left join nodes c
           on b.from = c.node
           left join nodes d
           on b.to = d.node
           where b.weight > {fetch_cutoff} /* Subset to manage table size */
           ;
        """
    )

    link_prediction_tbl = s.CASTable("LINKSRESJOINEDR")
    link_prediction_tbl["hit"] = link_prediction_tbl["type"]!=""
    return link_prediction_tbl

link_prediction_tbl = join_ground_truth(resembles_link_type, fetch_cutoff_weight, min_weight, max_weight)

NOTE: Table LINKSRESJOINEDR was created in caslib myData with 1090 rows returned.


In [18]:
def hits_at(n):
    tbl = link_prediction_tbl.nlargest(n, "weight")
    return int(sum(tbl["hit"]))

Count the correctly predicted disease-resembles-disease links among the top 10

In [19]:
hits_at(10)

7

Count the correctly predicted disease-resembles-disease links among the top 100

In [20]:
hits_at(100)

64

Count the correctly predicted disease-resembles-disease links among the top 1000

In [21]:
hits_at(1000)

303

In [22]:
print(f"Number of DrD true positives in entire data set: {n_drd}")
print(f"Number of Disease node pairs in entire data set: {int(n_diseases*(n_diseases-1)/2)}")

Number of DrD true positives in entire data set: 543
Number of Disease node pairs in entire data set: 9316


Show the top ten disease-resembles-disease link predictions.

In [23]:
link_prediction_tbl.nlargest(10, "weight")

,from,type,to,weight,fromName,toName,hit
0,Disease::DOID:8577,DrD,Disease::DOID:8778,26.239297,ulcerative colitis,Crohn's disease,1.0
1,,,,16.694609,hypertension,coronary artery disease,0.0
2,Disease::DOID:7148,DrD,Disease::DOID:9074,16.095346,rheumatoid arthritis,systemic lupus erythematosus,1.0
3,Disease::DOID:1319,DrD,Disease::DOID:3070,14.908940,brain cancer,malignant glioma,1.0
4,Disease::DOID:1936,DrD,Disease::DOID:3393,14.070502,atherosclerosis,coronary artery disease,1.0
5,Disease::DOID:3312,DrD,Disease::DOID:5419,13.300299,bipolar disorder,schizophrenia,1.0
6,Disease::DOID:7147,DrD,Disease::DOID:7148,12.887492,ankylosing spondylitis,rheumatoid arthritis,1.0
7,,,,12.730336,prostate cancer,breast cancer,0.0
8,Disease::DOID:7148,DrD,Disease::DOID:8893,12.354446,rheumatoid arthritis,psoriasis,1.0
9,,,,12.313293,breast cancer,hematologic cancer,0.0


There are three false positives, but a cursory internet search shows that there are, in fact, meaningful links between *hypertension* and *coronary artery disease*. The same is true for *prostate cancer* and *breast cancer*. So even though these are reported as false positives, they may represent incompleteness of the data set rather than inaccuracy of the prediction.

## Repeat the analysis for compound link prediction

In [24]:
keep_type = "Compound"

In [25]:
resembles_link_type, fetch_cutoff_weight = get_parameters(keep_type)
do_projections(keep_type)
join_nodes_data()
min_weight, max_weight = get_min_max_weights()
link_prediction_tbl = join_ground_truth(resembles_link_type, fetch_cutoff_weight, min_weight, max_weight)

NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 755.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.06 (cpu: 0.29) seconds.
NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 390.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.06 (cpu: 0.28) seconds.
NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 21102.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.09 (cpu: 0.73) seconds.
NOTE: The number of nodes in the input graph is 47031.
NOTE: The number of links in the input graph is 18756.
NOTE: Processing network projection using 16 threads across 1 machines.
NOTE: Processing projection used 0.09 (cpu: 0.74) seconds.
NOTE: The number of nodes in the input g

Count the correctly predicted compound-resembles-compound links among the top 10

In [26]:
hits_at(10)

10

Count the correctly predicted compound-resembles-compound links among the top 100

In [27]:
hits_at(100)

54

Count the correctly predicted compound-resembles-compound links among the top 1000

In [28]:
hits_at(1000)

249

In [29]:
print(f"Number of CrC true positives in entire data set: {n_crc}")
print(f"Number of Compound node pairs in entire data set: {int(n_compounds*(n_compounds-1)/2)}")

Number of CrC true positives in entire data set: 6486
Number of Compound node pairs in entire data set: 1203576


Show the top ten compound-resembles-compound link predictions.

In [30]:
link_prediction_tbl.nlargest(10, "weight")

,from,type,to,weight,fromName,toName,hit
0,Compound::DB00445,CrC,Compound::DB00997,69.171230,Epirubicin,Doxorubicin,1.0
1,Compound::DB00443,CrC,Compound::DB01234,68.079416,Betamethasone,Dexamethasone,1.0
2,Compound::DB00215,CrC,Compound::DB01175,58.447223,Citalopram,Escitalopram,1.0
3,Compound::DB00755,CrC,Compound::DB00982,54.594170,Tretinoin,Isotretinoin,1.0
4,Compound::DB00523,CrC,Compound::DB00982,54.154831,Alitretinoin,Isotretinoin,1.0
5,Compound::DB00523,CrC,Compound::DB00755,52.945645,Alitretinoin,Tretinoin,1.0
6,Compound::DB01137,CrC,Compound::DB01165,47.053058,Levofloxacin,Ofloxacin,1.0
7,Compound::DB00338,CrC,Compound::DB00736,39.835781,Omeprazole,Esomeprazole,1.0
8,Compound::DB00694,CrC,Compound::DB00997,37.237415,Daunorubicin,Doxorubicin,1.0
9,Compound::DB00445,CrC,Compound::DB00694,35.497282,Epirubicin,Daunorubicin,1.0


For the task of drug compound link prediction, all 10 of the top-weighted predictions are correctly predicted links.

In [31]:
s.terminate()